In [1]:
# importacion de librerias 
import numpy as np
import pandas as pd
import math 

In [2]:
# quiero ver en que directorio estoy 
import os

print(os.getcwd())
# como voy hasta la carpeta de datos que se encuentra en TFG/data 
os.chdir('../data')

/Users/antoniomoraabos/Desktop/CUARTO_IMAT/TFG/src


In [3]:
# leemos los distintos archivos csv 
df_checkins = pd.read_csv('Checkins.csv')
df_Pois = pd.read_csv('POIs.csv')
df_cities = pd.read_csv('Cities.csv')

In [4]:
# definimos una fúncion para calcular distancias con latitudes y longitudes
def haversine(lat1, lon1, lat2, lon2):
    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c 

In [ ]:
# Vamos a asignar a cada uno de los POIs la ciudad a la que pertenece. Para ello, iremos iterando por cada uno de los POIs. 
# Para cada uno de ellos, obtendremos su country_code, y buscaremos en el dataframe de Cities las ciudades que tenga el mismo country_code. Despues 
# calcularemos la distancia entre el POI y cada una de las  ciudades y nos quedaremos con la ciudad más cercana. 

# Iteramos por cada uno de los POIs
for i in range(len(df_Pois)):
    # Obtenemos el país del POI
    country_code = df_Pois.iloc[i]['country_code']
    # Filtramos las ciudades que pertenecen al mismo país
    cities = df_cities[df_cities['country_code'] == country_code]
    # Creamos una lista vacía para guardar las distancias entre el POI y cada una de las ciudades
    distances = []
    # Iteramos por cada una de las ciudades
    for j in range(len(cities)):
        # Obtenemos la latitud y longitud del POI y de la ciudad
        lat1 = df_Pois.iloc[i]['latitude']
        lon1 = df_Pois.iloc[i]['longitude']
        lat2 = cities.iloc[j]['latitude']
        lon2 = cities.iloc[j]['longitude']
        # Calculamos la distancia entre el POI y la ciudad
        distance = haversine(lat1, lon1, lat2, lon2)
        # Guardamos la distancia en la lista de distancias
        distances.append(distance)
    # Obtenemos el índice de la ciudad más cercana
    nearest_city_index = np.argmin(distances)
    # Obtenemos el nombre de la ciudad más cercana
    nearest_city = cities.iloc[nearest_city_index]['city_name']
    # Guardamos el nombre de la ciudad más cercana en la lista de ciudades más cercanas
    df_Pois.at[i, 'nearest_city'] = nearest_city




In [5]:
# pasemoslo a un csv 
df_pois_nearest_city = pd.read_csv('POIs_with_nearest_city.csv')
df_pois_nearest_city.head()

,venue_id,latitude,longitude,category_name,country_code,nearest_city
0,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US,Brooklyn
1,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US,New York
2,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US,Brooklyn
3,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US,Boston
4,3fd66200f964a52000ee1ee3,39.933178,-75.159262,Sandwich Place,US,Philadelphia


In [6]:
df_checkins.head()

,user_id,venue_id,UTC_time,time_offset
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:006 +000 2012,240
1,190571,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180
2,221021,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240
3,66981,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300
4,21010,4c2b4e8a9a559c74832f0de2,Tue Apr 03 18:00:09 +0000 2012,240


In [7]:
# vamos a eliminar los checkins que son del tipo Residential Building (Apartment / Condo)
# unamos el dataframe de checkins con el de POIs para poder filtrar los checkins que son del tipo Residential Building (Apartment / Condo)
df_pois_chekins = pd.merge(df_checkins, df_pois_nearest_city, on='venue_id', how='left')
df_pois_chekins = df_pois_chekins[df_pois_chekins['category_name'] != 'Residential Building (Apartment / Condo)']
# elimina aquellos df_pois_chekins que contengan Home en la columna category_name
df_pois_chekins = df_pois_chekins[~df_pois_chekins['category_name'].str.contains('Home', na=False)]
df_pois_chekins.head()
# pasemoslo a un csv de nombre Checkins_without_Residential_Building.csv
df_pois_chekins.to_csv('Checkins_without_Residential_Building.csv', index=False)

In [8]:
# Vamos ahora a quedarnos unicamente con los checkins que se hayan hecho en las ciudades de London, New York, Tokyo 
# Para ello, vamos a filtrar el dtaframe de poiss_checkins por las ciudades de London, New York y Tokyo

cities = ['London', 'New York', 'Tokyo']
df_pois_chekins = df_pois_chekins[df_pois_chekins['nearest_city'].isin(cities)]
df_pois_chekins.head()

,user_id,venue_id,UTC_time,time_offset,latitude,longitude,category_name,country_code,nearest_city
2,221021,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240,40.748939,-73.992280,Coffee Shop,US,New York
45,262915,4aec9f4bf964a52091c921e3,Tue Apr 03 18:00:39 +0000 2012,60,51.498044,-0.090546,Pub,GB,London
62,49932,4b1d4b75f964a520560e24e3,Tue Apr 03 18:00:52 +0000 2012,-240,40.664266,-73.720107,Department Store,US,New York
171,209256,4a65102bf964a5203bc71fe3,Tue Apr 03 18:02:05 +0000 2012,-240,40.732322,-73.985359,Gym,US,New York
213,198359,49edce76f964a520f9671fe3,Tue Apr 03 18:02:32 +0000 2012,-240,40.755881,-73.985778,Office,US,New York


In [9]:
# vamos a castear la columna UTC_Time a datetite , el formato de UTC_time es 
# tres inicales del dia , tres inicales del mes, el numero del día, la hora en este formato 18:00:00 + 0000 + año , vamos a castearlo a datetime
df_pois_chekins['datetime'] = pd.to_datetime(df_pois_chekins['UTC_time'], format="%a %b %d %H:%M:%S %z %Y", errors='coerce')
df_pois_chekins.head()
# vamos a eliminar las columnas UTC_time y time_offset 
df_pois_chekins = df_pois_chekins.drop(columns=['UTC_time', 'time_offset'])
df_pois_chekins.head()



,user_id,venue_id,latitude,longitude,category_name,country_code,nearest_city,datetime
2,221021,4a85b1b3f964a520eefe1fe3,40.748939,-73.992280,Coffee Shop,US,New York,2012-04-03 18:00:08+00:00
45,262915,4aec9f4bf964a52091c921e3,51.498044,-0.090546,Pub,GB,London,2012-04-03 18:00:39+00:00
62,49932,4b1d4b75f964a520560e24e3,40.664266,-73.720107,Department Store,US,New York,2012-04-03 18:00:52+00:00
171,209256,4a65102bf964a5203bc71fe3,40.732322,-73.985359,Gym,US,New York,2012-04-03 18:02:05+00:00
213,198359,49edce76f964a520f9671fe3,40.755881,-73.985778,Office,US,New York,2012-04-03 18:02:32+00:00


In [10]:
import pandas as pd

# Supongamos que 'df' es tu DataFrame original
# Si lo lees desde un archivo: df = pd.read_csv('tu_archivo.csv')

# Asegurarse de que 'datetime' esté en formato datetime
df_pois_chekins['datetime'] = pd.to_datetime(df_pois_chekins['datetime'])

# Agrupar por 'user_id' y 'venue_id'
grouped = df_pois_chekins.groupby(['user_id', 'venue_id'])

# Crear un nuevo DataFrame con las agregaciones necesarias
new_df = grouped.agg({
    'latitude': 'first',       # Tomamos el primer valor, ya que son consistentes
    'longitude': 'first',
    'category_name': 'first',
    'country_code': 'first',
    'nearest_city': 'first',
    'datetime': 'max'          # Seleccionamos el datetime más reciente
}).reset_index()

# Agregar la columna 'score' con el conteo de ocurrencias
new_df['score'] = grouped.size().values

# Reordenar las columnas (opcional)
new_df = new_df[['user_id', 'venue_id', 'latitude', 'longitude', 'category_name', 'country_code', 'nearest_city', 'datetime', 'score']]

# Mostrar el resultado
print(new_df.head())
# comprobemso que el conteo de cada par user_id, venue_id es correcto
new_df.groupby(['user_id', 'venue_id']).size().reset_index(name='counts').head()
# cuenta el número de venues distintos que aparecen en el dataframe
new_df['venue_id'].nunique() 
# cuenta el número de usuarios distintos que aparecen en el dataframe
new_df['user_id'].nunique()


   user_id                  venue_id   latitude  longitude     category_name  \
0        1  43a52546f964a520532c1fe3  40.645089 -73.784523           Airport   
1        1  49c046f1f964a5205b551fe3  40.645578 -73.776433  Airport Terminal   
2        1  49ee5737f964a5204e681fe3  40.630903 -73.739945     Grocery Store   
3        1  4a26b3d7f964a520c57e1fe3  40.712020 -73.723733         Racetrack   
4        1  4b168664f964a520fcb923e3  40.690740 -73.685458       Coffee Shop   

  country_code nearest_city                  datetime  score  
0           US     New York 2012-05-08 18:56:30+00:00      1  
1           US     New York 2012-05-08 18:56:54+00:00      2  
2           US     New York 2012-05-07 17:28:29+00:00      1  
3           US     New York 2012-05-17 15:11:53+00:00      1  
4           US     New York 2012-05-27 20:31:41+00:00      1  


14565

In [11]:
# Creamos un df para cada una de las ciudades , London, New York, Tokyo
# ver nombre todas las columnas 
df_london = new_df[new_df['nearest_city'] == 'London']
df_new_york = new_df[new_df['nearest_city'] == 'New York']
df_tokyo = new_df[new_df['nearest_city'] == 'Tokyo']
# guardemoslos como csv 
df_london.to_csv('Checkins_London.csv', index=False)
df_new_york.to_csv('Checkins_New_York.csv', index=False)
df_tokyo.to_csv('Checkins_Tokyo.csv', index=False)
# muestra pro pantalla el número de usuarios que ha visitado más de una ciudad
lista_usuarios_londres = df_london['user_id'].unique()
lista_usuarios_nueva_york = df_new_york['user_id'].unique()
lista_usuarios_tokio = df_tokyo['user_id'].unique()
# comprobamos cuantos usuarios aparecen en más de una lista 
len(set(lista_usuarios_londres) & set(lista_usuarios_nueva_york))+len(set(lista_usuarios_londres) & set(lista_usuarios_tokio))+ len(set(lista_usuarios_nueva_york) & set(lista_usuarios_tokio))- len(set(lista_usuarios_londres) & set(lista_usuarios_nueva_york) & set(lista_usuarios_tokio))


302

In [12]:
# Vamos a cargar los tres csv de chekins asociados a cada una de las ciudades
df_london = pd.read_csv('Checkins_London.csv')
df_new_york = pd.read_csv('Checkins_New_York.csv')
df_tokyo = pd.read_csv('Checkins_Tokyo.csv')
# Vamos a cambiar los valores de las columnas de user_id y venue_id, a enteros que comienzan en 0 
def map_ids(df):
    # Crear un mapeo de user_id a enteros
    user_mapping = {user: idx for idx, user in enumerate(df['user_id'].unique())}
    # Reemplazar los user_id en el dataframe
    df['user_id'] = df['user_id'].map(user_mapping)
    # Crear un mapeo de venue_id a enteros
    venue_mapping = {venue: idx for idx, venue in enumerate(df['venue_id'].unique())}
    # Reemplazar los venue_id en el dataframe
    df['venue_id'] = df['venue_id'].map(venue_mapping)
    return df

df_london = map_ids(df_london)
df_new_york = map_ids(df_new_york)
df_tokyo = map_ids(df_tokyo)

# Guardamos los dataframes como csv
df_london.to_csv('Checkins_London.csv', index=False)
df_new_york.to_csv('Checkins_New_York.csv', index=False)
df_tokyo.to_csv('Checkins_Tokyo.csv', index=False)

In [13]:
# pon el número de venues distintos que hay en cada uno de los dataframes y sumalos 
df_london['venue_id'].nunique() + df_new_york['venue_id'].nunique() + df_tokyo['venue_id'].nunique()
# pon el número de users distintos que hay en cada uno de los dataframes y sumalos
df_london['user_id'].nunique() + df_new_york['user_id'].nunique() + df_tokyo['user_id'].nunique()

14867